In [108]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

import gensim
from gensim import corpora, models, similarities

import nltk
from nltk.stem.porter import PorterStemmer
from nltk.stem.lancaster import LancasterStemmer
from nltk.corpus import stopwords

from pymongo import MongoClient
from time import time
from collections import Counter, defaultdict
from textblob import TextBlob
import pandas as pd
import numpy as np
import operator
import re
import os
import pickle

# Load Resume

In [164]:
with open('BryantBiggs_mod.txt', 'r') as f:
    new_resume = f.read().strip()

In [165]:
new_resume

'BRYANT J. BIGGS\n                          3134 Avalon Way           *            Bloomingdale, New Jersey            *            bryantbiggs@gmail.com\n\nEDUCATION\nPurdue University - West Lafayette, IN\nBachelor of Science - M.E.T.\nReceived - December 19, 2009\t\t\t\nProgram GPA - 3.28/4.00\n\nArizona State University - Tempe, AZ\nBachelor of Science - Software Eng.\nExpected - February, 2017\t\t\nProgram GPA - 4.00/4.00\n\t    \n\nEXPERIENCE\nDATA SCIENTIST\n         Metis - New York City, NY. June, 2016 - Current\nAn immersive 12 week data science boot camp to learn and apply concepts such as data science, data mining, supervised (SVM, decision trees, random forests, KNN) and unsupervised (NLP, KMeans) machine learning, statistical inference, Bayesian statistics, regression analysis, data visualization, and cloud computing.\nTechnologies used/taught:\n  ? Python - Scikit-learn, SciPy, NumPy, Pandas, BeautifulSoup, Selenium, Matplotlib, Flask, Django\n  ? Javascript - D3.js, jQu

# City, State Abbreviation List

In [166]:
with open(r'pkl/cities.pkl', 'rb') as infile:
       cities = pickle.load(infile)

with open(r'pkl/abbr.pkl', 'rb') as infile:
       abbr = pickle.load(infile)

# Clean Text

In [167]:
# remove special characters
new_resume = re.sub(r'([-/*&%+();?.,@#$^])', ' ', new_resume)

cleaner = ['\xa0']
for c in cleaner:
    new_resume = new_resume.replace(c,'')

# remove multiple white spaces
new_resume = re.sub('\n',' ', new_resume)
new_resume = re.sub('\t',' ', new_resume)
new_resume = re.sub(r'''[^0-9a-zA-Z ]+''', '', new_resume)
new_resume = re.sub(' +',' ', new_resume)

In [168]:
new_resume

'BRYANT J BIGGS 3134 Avalon Way Bloomingdale New Jersey bryantbiggs gmail com EDUCATION Purdue University West Lafayette IN Bachelor of Science M E T Received December 19 2009 Program GPA 3 28 4 00 Arizona State University Tempe AZ Bachelor of Science Software Eng Expected February 2017 Program GPA 4 00 4 00 EXPERIENCE DATA SCIENTIST Metis New York City NY June 2016 Current An immersive 12 week data science boot camp to learn and apply concepts such as data science data mining supervised SVM decision trees random forests KNN and unsupervised NLP KMeans machine learning statistical inference Bayesian statistics regression analysis data visualization and cloud computing Technologies used taught Python Scikit learn SciPy NumPy Pandas BeautifulSoup Selenium Matplotlib Flask Django Javascript D3 js jQuery SQL and NoSQL databases PostgreSQL MongoDB Apache Hadoop MapReduce and Spark APPLICATION ENGINEER Saint Gobain Performance Plastics Wayne NJ June 2014 Current Product design for PTFE based

# Remove StopWords

In [169]:
# cache stopwords first to reduce compute time
cachedStopWords = stopwords.words("english")
cachedStopWords += ['tot']
cachedStopWords += cities
cachedStopWords += abbr
cachedStopWords = list(set(cachedStopWords))

In [170]:
# convert all text to lower case and separate into list
res_text = [x for x in new_resume.lower().split()]

# remove stopwords
res_text = ' '.join([item for item in res_text if item not in cachedStopWords])
print(len(res_text.split()))

300


In [171]:
res_text

'j 3134 bryantbiggs gmail com education bachelor e received december 19 2009 program gpa 3 28 4 00 bachelor software eng expected february 2017 program gpa 4 00 4 00 experience data scientist metis 2016 current immersive 12 week data boot learn apply concepts data data mining supervised svm decision trees forests knn unsupervised nlp kmeans machine learning statistical inference bayesian statistics regression analysis data visualization computing technologies used taught python scikit learn scipy numpy pandas beautifulsoup selenium matplotlib flask django javascript d3 js jquery sql nosql databases postgresql mongodb hadoop mapreduce spark application engineer gobain performance plastics nj 2014 current product ptfe based bearings used primarily automotive performance bicycle applications product application development initial prototype launch within global engineering team development applications assist engineering drawing document creation automated script create based recommendati

# More Cleaning (Ugly - Fix Me)

In [172]:
res_text = res_text.replace('aacademic', 'academic')
res_text = res_text.replace('aaccounts', 'account')
res_text = res_text.replace('aaerobic', 'aerobic')
res_text = res_text.replace('aademy', 'academy')

res_text = res_text.replace('aanalyst', 'analyst')
res_text = res_text.replace('aanalyzed', 'analyst')

res_text = res_text.replace('reports', 'report')
res_text = res_text.replace('reporting', 'report')

res_text = res_text.replace('engineering', 'engineer')
res_text = res_text.replace('engineers', 'engineer')

res_text = res_text.replace('services', 'service')
res_text = res_text.replace('servicing', 'service')

res_text = res_text.replace('systems', 'system')

res_text = res_text.replace('abandoned', 'abandon')
res_text = res_text.replace('abandoner', 'abandon')
res_text = res_text.replace('abandoning', 'abandon')
res_text = res_text.replace('abandonment', 'abandon')
res_text = res_text.replace('abandons', 'abandon')

res_text = res_text.replace('management', 'manage')
res_text = res_text.replace('manager', 'manage')

res_text = res_text.replace('development', 'develop')
res_text = res_text.replace('developer', 'develop')

print(len(res_text.split()))
res_text

300


'j 3134 bryantbiggs gmail com education bachelor e received december 19 2009 program gpa 3 28 4 00 bachelor software eng expected february 2017 program gpa 4 00 4 00 experience data scientist metis 2016 current immersive 12 week data boot learn apply concepts data data mining supervised svm decision trees forests knn unsupervised nlp kmeans machine learning statistical inference bayesian statistics regression analysis data visualization computing technologies used taught python scikit learn scipy numpy pandas beautifulsoup selenium matplotlib flask django javascript d3 js jquery sql nosql databases postgresql mongodb hadoop mapreduce spark application engineer gobain performance plastics nj 2014 current product ptfe based bearings used primarily automotive performance bicycle applications product application develop initial prototype launch within global engineer team develop applications assist engineer drawing document creation automated script create based recommendation document cu

# Word Counts

In [173]:
def get_wordcount(text_list, min_ct=3, most_common=30, get_all=False):
    '''
    returns => most common
    '''
    # get wordcount counter object
    word_count = Counter(text_list)

    # remove words that occur min_ct times or less
    word_count = Counter({k:v for k, v in word_count.items() if v >= min_ct})

    if get_all:
        # return all
        word_count = word_count.items()
    else:
        # limit wordcounts for visualization
        word_count = word_count.most_common(most_common)
    
    return word_count

### Extrack Noun Phrases

In [174]:
res_nouns = TextBlob(res_text).noun_phrases
res_nouns

WordList(['bryantbiggs gmail com education bachelor e', 'program gpa', 'bachelor software eng', 'program gpa', 'experience data scientist metis', 'current immersive', 'week data boot', 'concepts data data', 'svm decision trees forests knn', 'nlp kmeans machine learning', 'statistical inference bayesian statistics regression analysis data visualization', 'python scikit', 'scipy numpy pandas beautifulsoup selenium matplotlib flask django javascript d3 js jquery sql nosql databases postgresql mongodb hadoop mapreduce', 'application engineer gobain performance plastics nj', 'current product ptfe', 'automotive performance bicycle applications product application', 'initial prototype launch', 'global engineer team', 'engineer drawing document creation', 'recommendation document customers material usage calculator inventor', 'script automate detail drawings macro', '3d parametric models', 'internal external tool', 'multiple access database', 'hr team review employee performance metrics career

### Merge Noun Phrases Back to Text String

In [175]:
temp = []
for i in res_nouns:
    temp.append(i)

res_nouns = ''
for x in temp:
    res_nouns += x + ' '
res_nouns

'bryantbiggs gmail com education bachelor e program gpa bachelor software eng program gpa experience data scientist metis current immersive week data boot concepts data data svm decision trees forests knn nlp kmeans machine learning statistical inference bayesian statistics regression analysis data visualization python scikit scipy numpy pandas beautifulsoup selenium matplotlib flask django javascript d3 js jquery sql nosql databases postgresql mongodb hadoop mapreduce application engineer gobain performance plastics nj current product ptfe automotive performance bicycle applications product application initial prototype launch global engineer team engineer drawing document creation recommendation document customers material usage calculator inventor script automate detail drawings macro 3d parametric models internal external tool multiple access database hr team review employee performance metrics career growth product engineer wyman december process models heat treatment fixtures fin

### Tokenize the documents, remove stop words and words that only appear once

In [176]:
# remove common words and tokenize
stoplist = set('for a of the and to in'.split())
texts = [word for word in res_nouns.split()]

# remove words that appear only once
frequency = defaultdict(int)
for text in texts:
        frequency[text] += 1

# remove words that occur less than n times
texts = [token for token in texts if frequency[token] > 1]

In [177]:
str_text = ' '.join(texts)

# Load Transformation Interface

In [178]:
# load sparse vector matrix
if (os.path.exists('pkl/resume_token.mm')):
    corpus = corpora.MmCorpus('pkl/resume_token.mm')
    print('Sparse matrix LOADED as \'corpus\'')
else:
    print('Sparse matrix NOT FOUND')

Sparse matrix LOADED as 'corpus'


In [179]:
# load tokenized dictionary
if (os.path.exists('pkl/resume_token.dict')):
    dictionary = corpora.Dictionary.load('pkl/resume_token.dict')
    print('Tokenized dictionary LOADED as \'dictionary\'')
else:
    print('Tokenized dictionary NOT FOUND')

Tokenized dictionary LOADED as 'dictionary'


# Load Trained Model

In [180]:
lsi_mdl = models.LsiModel.load('pkl/lsi_mdl.lsi')

In [181]:
vec_bow = dictionary.doc2bow(texts)
vec_lsi = lsi_mdl[vec_bow] # convert the query to LSI space
print(vec_lsi)

[(0, 4.1890737721615015), (1, 0.023039548341981873), (2, 1.7245138544616001), (3, -0.35117028649083498), (4, 2.0275665799753484), (5, -0.33443548539765361), (6, 0.54307590115017113), (7, 1.679951413953862), (8, -1.189282438962171), (9, -3.0291138152702439), (10, 0.057594797230231691), (11, -0.16112496969211593), (12, -0.90883873691300032), (13, 0.46083935498708828), (14, 0.13334925572578391), (15, -2.4113033651482789), (16, 0.10934954196576055), (17, 1.1764494617618728), (18, 0.13858930567324715), (19, 0.082570584246127593), (20, 0.78933292264673038), (21, -0.34207711531897078), (22, 0.0014776902816088944), (23, -0.71325280063489049), (24, -0.82571080458009027), (25, 0.34032624791648758), (26, 1.9161529822969192), (27, -0.25108389900440498), (28, 0.25666584339334814), (29, 0.24004422994842953), (30, 0.38877242426613917), (31, -0.32009701564467935), (32, 0.14222161328227897), (33, -0.090667467335837498), (34, 0.23264500467028387), (35, 0.28350715496686996), (36, 0.83858248915488365), (3

In [182]:
index = similarities.MatrixSimilarity(lsi_mdl[corpus])

In [185]:
# perform a similarity query against the corpus
sims = index[vec_lsi] 

# (document_number, document_similarity)
sim_lst = list(enumerate(sims))

# sort relevant documents first
sim_lst.sort(key=operator.itemgetter(1), reverse=False)

In [186]:
sim_lst[1:6]

[(3955, -0.0089570619),
 (16152, -0.0034703016),
 (16033, -0.00082496554),
 (15972, 0.0034999102),
 (16534, 0.0091814548)]

In [140]:
df = pd.read_pickle('pkl/df_stop_noun.pkl')
df.iloc[4462]['resume_nouns']

'hower chief engineer bore current product engineer bereid overal naartoe te verhuizen chief engineer bore current product engineer navistar inc engine engineer engineer efforts current product warranty cost reduction supplier requests navistar 11l 13l 15l bore engines immediate staff direction priorities engineer test organization warranty cost r process warranty diagnosis changes functional team important engine engine consumption control 13l engines project validation production implementation inventory control product changes root causes solutions multiple product performance warranty issues oversight participation 8d problem teams implementation solutions elimination warranty claims chief engineer navistar defense engine oem engine opportunities navistar inc engine engineer organization identify pursue opportunities navistar engines oem commercial military vehicle applications technical specifications analysis opportunities conversion dt570 i6 commercial truck engine military resi

In [141]:
df.iloc[485]['resume_nouns']

'ritt materials engineer orthodontics hands product substantial experience multiple projects stringent quality system functional teams sophisticated analytical instruments bereid overal naartoe te verhuizen materials engineer orthodontics januari heden product engineer engineer project manage team leader project manage product process concept definition introduction necessary activities analysis reviews risk analysis verification validation requirements specifications test methods protocols significant experience erp pdm system drawings numbers descriptions boms routings engineer support product lines product life cycle introduction extensions discontinuation superfluous product continuous improvement process changes engineer expertise quality product updates drawings specifications processes procedures engineer analysis product customer complaints assistant aalto espoo juni espoo finland defects indium nitride journal physics conference series metallurgy intern luvata llc maart variou

In [142]:
df.iloc[149]['resume_nouns']

'dirks project engineer project engineer mechanical engineer responsibility mechanical functionality form factor product electromechanical current portfolio expertise project r 3d models fea mfa component correspondence venture suppliers quality activities capacitor product engineer manage correspondence suppliers engineer production requirements quality activities efforts double digit sales increase years travel regular intervals product coordinator testors rust oleum company volume automotive touch system product oem supplier wal diy automotive stores product projects work flow lean initiative reason engineer department senior product engineer multiple mechanical engineer activities battery chemistry startup company caterpillar inc r group form facture engineer cad fea complete battery storage enclosure inner structure complete product reason series c economic turn overseas senior engineer textron company mechanical engineer activates test measurement unit multiple domestic locations